In [ ]:
import gradio as gr
from PIL import Image
from ultralytics import YOLO
from deep_translator import GoogleTranslator
from gtts import gTTS
import tempfile

In [ ]:
# โหลด YOLOv8
model = YOLO("yolov8n.pt")  # เลือก model เล็กสุด ใช้งานเร็ว

# ภาษา
languages = {
    "Thai": "th", "English": "en", "Korean": "ko", "Japanese": "ja",
    "Chinese": "zh-CN", "Russian": "ru", "French": "fr",
    "Vietnamese": "vi", "Spanish": "es", "Portuguese": "pt"
}


In [ ]:
# ตรวจจับวัตถุหลักจากภาพด้วย YOLO
def detect_main_object(image):
    results = model(image)[0]
    if len(results.boxes) == 0:
        return None
    best_box = max(results.boxes, key=lambda box: box.conf.cpu().item())
    class_id = int(best_box.cls.cpu().item())
    label = model.model.names[class_id]
    return label

# แปลคำ
def translate_word(word, lang_code):
    try:
        return GoogleTranslator(source="auto", target=lang_code).translate(word)
    except Exception as e:
        print("Translation error:", e)
        return "(แปลไม่ได้)"

# สร้างเสียง
def speak_word(text, lang_code):
    try:
        tts = gTTS(text=text, lang=lang_code)
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmp:
            tts.save(tmp.name)
            return tmp.name
    except Exception as e:
        print("TTS error:", e)
        return None

# ทำนายภาพ

def predict(image, selected_langs):
    pred_label = detect_main_object(image)
    if not pred_label:
        return ["ไม่พบวัตถุเด่นในภาพ"] + ["" for _ in languages] + [gr.update(visible=False) for _ in languages]

    outputs = {"label": pred_label}
    for lang in languages:
        if lang in selected_langs:
            translated = translate_word(pred_label, languages[lang])
            audio_path = speak_word(translated, languages[lang])
            outputs[lang] = (translated, audio_path)
        else:
            outputs[lang] = ("", None)

    return [outputs["label"]] + [outputs[lang][0] for lang in languages] + [gr.update(value=outputs[lang][1], visible=bool(outputs[lang][1])) for lang in languages]

In [ ]:
# UI
with gr.Blocks() as app:
    gr.Markdown("""
    # 📸 SnapTranslate by Pawit
    📲 อัปโหลดภาพ → ทำนายวัตถุหลักด้วย YOLO → แปลหลายภาษา → ฟังเสียง
    """)

    with gr.Row():
        image_input = gr.Image(type="pil", label="📄 อัปโหลดรูปภาพ")
        lang_select = gr.CheckboxGroup(choices=list(languages.keys()), label="🌐 เลือกภาษาที่ต้องการแปล")

    predict_btn = gr.Button("🧐 แปลภาพ")
    prediction_text = gr.Textbox(label="🔍 วัตถุหลักในภาพ (ภาษาอังกฤษ)")

    output_texts = {}
    audio_outputs = {}

    for lang in languages:
        with gr.Row():
            output_texts[lang] = gr.Textbox(label=f"{lang}:", visible=True)
            audio_outputs[lang] = gr.Audio(label="", visible=False)

    predict_btn.click(
        fn=predict,
        inputs=[image_input, lang_select],
        outputs=[prediction_text] + list(output_texts.values()) + list(audio_outputs.values())
    )

app.launch(debug=True, share=True)